# Transformer model architecture
![](./images/transformer_architecture.png)

# Transformer implementation

In [1]:
import torch

## Embed

### Embedding

In [2]:
class Embedder(torch.nn.Module):
    
    def __init__(self, vocab_size, d_model):
        super().__init__()
        self.embed = torch.nn.Embedding(vocab_size, d_model)
        
    def forward(self, x):
        # [123, 0, 23, 5] -> [[..512..], [...512...], ...]
        return self.embed(x)

### Positional encoding

$$ PE_{(pos, 2i)} = sin(pos/10000^{2i/d_{model}}) $$

$$ PE_{(pos, 2i + 1)} = cos(pos/10000^{2i/d_{model}}) $$

In [3]:
import math

class PositionalEncoder(torch.nn.Module):
    
    def __init__(self, d_model, max_seq_len=80):
        super().__init__()
        self.d_model = d_model
        
        # create constant positional encoding matrix
        pe_matrix = torch.zeros(max_seq_len, d_model)
        
        for pos in range(max_seq_len):
            for i in range(0, d_model, 2):
                pe_matrix[pos, i] = math.sin(pos/10000**(2*i/d_model))
                pe_matrix[pos, i+1] = math.cos(pos/10000**(2*i/d_model))
        pe_matrix = pe_matrix.unsqueeze(0)     # Add one dimension for batch size
        self.register_buffer('pe', pe_matrix)  # Register as persistent buffer
        
    def forward(self, x):
        # x is a sentence after embedding with dim (batch, number of words, vector dimension)
        seq_len = x.size()[1]
        x = x + self.pe[:, :seq_len]
        return x

## Model layers

### Scaled Dot-Product Attention layer

![](images/scaled_dot_product_attention.png)

12*512

[512..] x M = k_0
[...512] x M = k_1

In [4]:
import math
import torch.nn.functional as F

# Given Query, Key, Value, calculate the final weighted value
def scaled_dot_product_attention(q, k, v, mask=None, dropout=None):
    # Shape of q and k are the same, both are (batch_size, seq_len, d_k)
    # Shape of v is (batch_size, seq_len, d_v)
    attention_scores = torch.matmul(q, k.transpose(-2, -1))/math.sqrt(q.shape[-1])  # size (batch_size, seq_len, seq_len)
    
    # Apply mask to scores
    # <pad>
    if mask is not None:
        attention_scores = attention_scores.masked_fill(mask == 0, value=-1e9)
        
    # Softmax along the last dimension
    attention_weights = F.softmax(attention_scores, dim=-1)
    
    if dropout is not None:
        attention_weights = dropout(attention_weights)
        
    output = torch.matmul(attention_weights, v)
    return output

### Multi-Head Attention layer

![](images/multi_head_attention.png)

In [5]:
class MultiHeadAttention(torch.nn.Module):
    def __init__(self, n_heads, d_model, dropout=0.1):
        super().__init__()
        
        self.n_heads = n_heads
        self.d_model = d_model
        self.d_k = self.d_v = d_model//n_heads
        
        # self attention linear layers
        # Linear layers for q, k, v vectors generation in different heads
        self.q_linear_layers = []
        self.k_linear_layers = []
        self.v_linear_layers = []
        for i in range(n_heads):
            self.q_linear_layers.append(torch.nn.Linear(d_model, self.d_k))
            self.k_linear_layers.append(torch.nn.Linear(d_model, self.d_k))
            self.v_linear_layers.append(torch.nn.Linear(d_model, self.d_v))
        
        self.dropout = torch.nn.Dropout(dropout)
        self.out = torch.nn.Linear(n_heads*self.d_v, d_model)
        
    def forward(self, q, k, v, mask=None):
        multi_head_attention_outputs = []
        for q_linear, k_linear, v_linear in zip(self.q_linear_layers,
                                                self.k_linear_layers,
                                                self.v_linear_layers):
            new_q = q_linear(q)  # size: (batch_size, seq_len, d_k)
            new_k = k_linear(k)  # size: (batch_size, seq_len, d_k)
            new_v = v_linear(v)  # size: (batch_size, seq_len, d_v)
            
            # Scaled Dot-Product attention
            head_v = scaled_dot_product_attention(new_q, new_k, new_v, mask, self.dropout)  # (batch_size, seq_len, d_v)
            multi_head_attention_outputs.append(head_v)
            
        # Concat
        #import pdb; pdb.set_trace()
        concat = torch.cat(multi_head_attention_outputs, -1)  # (batch_size, seq_len, n_heads*d_v)
        
        # Linear layer to recover to original shap
        output = self.out(concat)  # (batch_size, seq_len, d_model)
        
        return output

### Feed Forward layer

In [6]:
class FeedForward(torch.nn.Module):
    def __init__(self, d_model, d_ff=2048, dropout=0.1):
        super().__init__()
        
        self.linear_1 = torch.nn.Linear(d_model, d_ff)
        self.dropout = torch.nn.Dropout(dropout)
        self.linear_2 = torch.nn.Linear(d_ff, d_model)
        
    def forward(self, x):
        x = self.dropout(F.relu(self.linear_1(x)))
        x = self.linear_2(x)
        return x

### Layer Normalization layer

#### Normalization

$$\mu = \frac{1}{m} \sum_{i=1}^{m}x_i$$

$$
\sigma^{2} = \frac{1}{m} \sum^{m}_{i=1}(x_{i} - \mu)^2
$$

$$
\hat{Z}_i = \frac{x_i - \mu_i}{\sqrt{\sigma^{2}_{i} + \epsilon}}
$$

#### Add two learnable parameters

$$
\tilde{Z}_i = \alpha_i * \hat{Z}_i + \beta_i
$$

In [7]:
class LayerNorm(torch.nn.Module):
    def __init__(self, d_model, eps=1e-6):
        super().__init__()
        self.d_model = d_model
        self.alpha = torch.nn.Parameter(torch.ones(self.d_model))
        self.beta = torch.nn.Parameter(torch.zeros(self.d_model))
        self.eps = eps
        
    def forward(self, x):
        # x size: (batch_size, seq_len, d_model)
        x_hat = (x - x.mean(dim=-1, keepdim=True))/(x.std(dim=-1, keepdim=True) + self.eps)
        x_tilde = self.alpha*x_hat + self.beta
        return x_tilde

## Encoder & Decoder layers

### Encoder layer

An encoder layer contains a multi-head attention layer and feed forward layer

![](images/encoder.png)

In [8]:
class EncoderLayer(torch.nn.Module):
    def __init__(self, d_model, n_heads, dropout=0.1):
        super().__init__()
        self.d_model = d_model
        self.norm_1 = LayerNorm(d_model)
        self.norm_2 = LayerNorm(d_model)
        self.multi_head_attention = MultiHeadAttention(n_heads, d_model)
        self.feed_forward = FeedForward(d_model)
        self.dropout_1 = torch.nn.Dropout(dropout)
        self.dropout_2 = torch.nn.Dropout(dropout)
        
    def forward(self, x, mask):
        #import pdb; pdb.set_trace()
        x = x + self.dropout_1(self.multi_head_attention(x, x, x, mask))
        x = self.norm_1(x)
        
        x = x + self.dropout_2(self.feed_forward(x))
        x = self.norm_2(x)
        return x

### Decoder layer

An decoder layer contains two multi-head attention layers and one feed forward layer

![](images/decoder.png)

In [9]:
class DecoderLayer(torch.nn.Module):
    def __init__(self, d_model, n_heads, dropout=0.1):
        super().__init__()
        self.norm_1 = LayerNorm(d_model)
        self.norm_2 = LayerNorm(d_model)
        self.norm_3 = LayerNorm(d_model)
        
        self.dropout_1 = torch.nn.Dropout(dropout)
        self.dropout_2 = torch.nn.Dropout(dropout)
        self.dropout_3 = torch.nn.Dropout(dropout)
        
        self.multi_head_attention_1 = MultiHeadAttention(n_heads, d_model)
        self.multi_head_attention_2 = MultiHeadAttention(n_heads, d_model)
        
        self.feed_forward = FeedForward(d_model)
        
    def forward(self, x, encoder_output, src_mask, trg_mask):
        x = self.dropout_1(self.multi_head_attention_1(x, x, x, trg_mask))
        x = x + self.norm_1(x)
        
        x = self.dropout_2(self.multi_head_attention_2(x, encoder_output, encoder_output, src_mask))
        x = x + self.norm_2(x)
        
        x = self.dropout_3(self.feed_forward(x))
        x = x + self.norm_3(x)
        
        return x

In [10]:
import copy

def clone_layer(module, N):
    return torch.nn.ModuleList([copy.deepcopy(module) for i in range(N)])

## Encoder & Decoder

In [11]:
class Encoder(torch.nn.Module):
    def __init__(self, vocab_size, d_model, N, n_heads):
        super().__init__()
        self.embed = Embedder(vocab_size, d_model)
        self.pe = PositionalEncoder(d_model)
        self.encoder_layers = clone_layer(EncoderLayer(d_model, n_heads), N)
        self.norm = LayerNorm(d_model)
        
    def forward(self, src, mask):
        x = self.embed(src)
        x = self.pe(x)
        for encoder in self.encoder_layers:
            x = encoder(x, mask)
        return self.norm(x)

In [12]:
class Decoder(torch.nn.Module):
    def __init__(self, vocab_size, d_model, N, n_heads):
        super().__init__()
        self.embed = Embedder(vocab_size, d_model)
        self.pe = PositionalEncoder(d_model)
        self.decoder_layers = clone_layer(DecoderLayer(d_model, n_heads), N)
        self.norm = LayerNorm(d_model)
        
    def forward(self, trg, encoder_output, src_mask, trg_mask):
        x = self.embed(trg)
        x = self.pe(x)
        for decoder in self.decoder_layers:
            x = decoder(x, encoder_output, src_mask, trg_mask)
        return self.norm(x)

## Transformer

In [13]:
class Transformer(torch.nn.Module):
    def __init__(self, src_vocab_size, trg_vocab_size, d_model, N, n_heads):
        super().__init__()
        self.encoder = Encoder(src_vocab_size, d_model, N, n_heads)
        self.decoder = Decoder(trg_vocab_size, d_model, N, n_heads)
        self.linear = torch.nn.Linear(d_model, trg_vocab_size)
        
    def forward(self, src, trg, src_mask, trg_mask):
        encoder_output = self.encoder(src, src_mask)
        decoder_output = self.decoder(trg, encoder_output, src_mask, trg_mask)
        output = self.linear(decoder_output)
        return output

# Data processing

In [14]:
import spacy
from torchtext import data

In [15]:
nlp = spacy.load('pt_core_news_sm')

tokenizer = lambda sentence: [tok.text for tok in nlp.tokenizer(sentence) if tok.text != " "]

In [16]:
SRC = data.Field(lower=True, tokenize=tokenizer)
TRG = data.Field(lower=True, tokenize=tokenizer, init_token='<sos>', eos_token='<eos>')

In [17]:
src_data = open('data/english.txt', 'r')

In [18]:
trg_data = open('data/french.txt', 'r')

In [19]:
raw_data = {'src': [line for line in src_data], 'trg': [line for line in trg_data]}

In [20]:
import pandas as pd

In [21]:
df = pd.DataFrame(raw_data, columns=['src', 'trg'])

In [22]:
df

,src,trg
0,Go.\n,Va !\n
1,Run!\n,Cours !\n
2,Run!\n,Courez !\n
3,Fire!\n,Au feu !\n
4,Help!\n,À l'aide !\n
5,Jump.\n,Saute.\n
6,Stop!\n,Ça suffit !\n
7,Stop!\n,Stop !\n
8,Stop!\n,Arrête-toi !\n
9,Wait!\n,Attends !\n


In [23]:
df.to_csv('en_to_fr.csv', index=False)

In [24]:
ls

README.md                en_to_fr.csv             test.ipynb
data/                    images/                  transformer_model.ipynb


In [25]:
data_fields = [('src', SRC), ('trg', TRG)]

In [26]:
train_set = data.TabularDataset('./en_to_fr.csv', format='csv', fields=data_fields)

In [27]:
SRC.build_vocab(train_set)

In [28]:
len(SRC.vocab)

14114

In [29]:
TRG.build_vocab(train_set)

In [30]:
len(TRG.vocab)

28353

In [31]:
train_set

# Train transformer

In [32]:
# set some parameters
d_model = 512
n_heads = 8
N = 6
src_vocab_size = len(SRC.vocab)
trg_vocab_size = len(TRG.vocab)

In [33]:
model = Transformer(src_vocab_size, trg_vocab_size, d_model, N, n_heads)

In [34]:
for p in model.parameters():
    if p.dim() > 1:
        torch.nn.init.xavier_uniform(p)

/Users/zjshao/.pyenv/versions/3.5.3/envs/ml/lib/python3.5/site-packages/ipykernel_launcher.py:3: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  This is separate from the ipykernel package so we can avoid doing imports until


In [35]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

In [36]:
train_iter = data.Iterator(train_set, batch_size=32, sort_key=lambda x: (len(x.src), len(x.trg)), shuffle=True, train=True)

In [37]:
import numpy as np

def create_mask(src_input, trg_input):
    # Source input mask
    pad = SRC.vocab.stoi['<pad>']
    src_mask = (src_input != pad).unsqueeze(1)
    
    # Target input mask
    trg_mask = (trg_input != pad).unsqueeze(1)
    
    seq_len = trg_input.size(1)
    nopeak_mask = np.tril(np.ones((1, seq_len, seq_len)), k=0).astype('uint8')
    nopeak_mask = torch.from_numpy(nopeak_mask) != 0
    trg_mask = trg_mask & nopeak_mask
    
    return src_mask, trg_mask

In [38]:
import time

def train_model(n_epochs, output_interval=100):
    model.train()
    start = time.time()
    
    for epoch in range(n_epochs):
        
        total_loss = 0
        for i, batch in enumerate(train_iter):
            
            src_input = batch.src.transpose(0, 1)  # size (batch_size, seq_len)
            trg = batch.trg.transpose(0, 1)  # size (batch_size, seq_len)
            
            trg_input = trg[:, :-1]
            ys = trg[:, 1:].contiguous().view(-1)
            
            # create src & trg masks
            src_mask, trg_mask = create_mask(src_input, trg_input)
            preds = model(src_input, trg_input, src_mask, trg_mask)
            
            optimizer.zero_grad()
            loss = F.cross_entropy(preds.view(-1, preds.size(-1)), ys, ignore_index=1)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.data[0]

            if (i + 1) % output_interval == 0:
                avg_loss = total_loss/output_interval
                print('time = {}, epoch = {}, iter = {}, loss = {}'.format((time.time() - start)/60,
                                                                           epoch + 1,
                                                                           i + 1,
                                                                           avg_loss))
                total_loss = 0
                start = time.time()

In [39]:
train_model(3, output_interval=1)

/Users/zjshao/.pyenv/versions/3.5.3/envs/ml/lib/python3.5/site-packages/ipykernel_launcher.py:27: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


time = 0.07368873357772827, epoch = 1, iter = 1, loss = 10.275301933288574
time = 0.05243396759033203, epoch = 1, iter = 2, loss = 10.036077499389648
time = 0.05758536656697591, epoch = 1, iter = 3, loss = 9.876063346862793
time = 0.059804431597391766, epoch = 1, iter = 4, loss = 9.840919494628906
time = 0.06737850109736125, epoch = 1, iter = 5, loss = 9.733710289001465
time = 0.060716116428375246, epoch = 1, iter = 6, loss = 9.621753692626953
time = 0.06828871568044027, epoch = 1, iter = 7, loss = 9.605151176452637
time = 0.06440561612447103, epoch = 1, iter = 8, loss = 9.512434959411621
time = 0.06431623299916585, epoch = 1, iter = 9, loss = 9.478424072265625
time = 0.06724570194880168, epoch = 1, iter = 10, loss = 9.49864387512207
time = 0.058811533451080325, epoch = 1, iter = 11, loss = 9.33305835723877
time = 0.061342501640319826, epoch = 1, iter = 12, loss = 9.282251358032227
time = 0.07716486851374309, epoch = 1, iter = 13, loss = 9.268787384033203
time = 0.06068573395411173, ep

KeyboardInterrupt: 